In [63]:
import requests
import os
from dotenv import load_dotenv
import logging

load_dotenv()
logging.basicConfig(level=logging.DEBUG)
logger = logging.getLogger(__name__)

uri = 'https://adventofcode.com/2025/day/1/input'
input = requests.get(uri, cookies={"session": os.getenv("SESSION_TOKEN")}).text

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): adventofcode.com:443
DEBUG:urllib3.connectionpool:https://adventofcode.com:443 "GET /2025/day/1/input HTTP/1.1" 200 6885


Day 1.

--- Day 1: Secret Entrance ---

The Elves have good news and bad news.

The good news is that they've discovered project management! This has given them the tools they need to prevent their usual Christmas emergency. For example, they now know that the North Pole decorations need to be finished soon so that other critical tasks can start on time.

The bad news is that they've realized they have a different emergency: according to their resource planning, none of them have any time left to decorate the North Pole!

To save Christmas, the Elves need you to finish decorating the North Pole by December 12th.

Collect stars by solving puzzles. Two puzzles will be made available on each day; the second puzzle is unlocked when you complete the first. Each puzzle grants one star. Good luck!

You arrive at the secret entrance to the North Pole base ready to start decorating. Unfortunately, the password seems to have been changed, so you can't get in. A document taped to the wall helpfully explains:

"Due to new security protocols, the password is locked in the safe below. Please see the attached document for the new combination."

The safe has a dial with only an arrow on it; around the dial are the numbers 0 through 99 in order. As you turn the dial, it makes a small click noise as it reaches each number.

The attached document (your puzzle input) contains a sequence of rotations, one per line, which tell you how to open the safe. A rotation starts with an L or R which indicates whether the rotation should be to the left (toward lower numbers) or to the right (toward higher numbers). Then, the rotation has a distance value which indicates how many clicks the dial should be rotated in that direction.

So, if the dial were pointing at 11, a rotation of R8 would cause the dial to point at 19. After that, a rotation of L19 would cause it to point at 0.

Because the dial is a circle, turning the dial left from 0 one click makes it point at 99. Similarly, turning the dial right from 99 one click makes it point at 0.

So, if the dial were pointing at 5, a rotation of L10 would cause it to point at 95. After that, a rotation of R5 could cause it to point at 0.

The dial starts by pointing at 50.

You could follow the instructions, but your recent required official North Pole secret entrance security training seminar taught you that the safe is actually a decoy. The actual password is the number of times the dial is left pointing at 0 after any rotation in the sequence.

For example, suppose the attached document contained the following rotations:

L68

L30

R48

L5

R60

L55

L1

L99

R14

L82

Following these rotations would cause the dial to move as follows:

The dial starts by pointing at 50.

The dial is rotated L68 to point at 82.

The dial is rotated L30 to point at 52.

The dial is rotated R48 to point at 0.

The dial is rotated L5 to point at 95.

The dial is rotated R60 to point at 55.

The dial is rotated L55 to point at 0.

The dial is rotated L1 to point at 99.

The dial is rotated L99 to point at 0.

The dial is rotated R14 to point at 14.

The dial is rotated L82 to point at 32.


Because the dial points at 0 a total of three times during this process, the password in this example is 3.

Analyze the rotations in your attached document. What's the actual password to open the door?

In [53]:
def zeroes_count(start, numbers, target):
    current = start
    num_of_occurrences = 0
    for next in numbers:
        current += next
        if current % 100 == target:
            num_of_occurrences += 1
    return num_of_occurrences


In [72]:
input_split = input.splitlines()
input_numbers = [int(x[1:]) if x[0] == 'R' else -int(x[1:]) for x in input_split]
solution = zeroes_count(50, input_numbers, 0)
print(f"The solution is {solution}.")

The solution is 1059.


--- Part Two ---

You're sure that's the right password, but the door won't open. You knock, but nobody answers. You build a snowman while you think.

As you're rolling the snowballs for your snowman, you find another security document that must have fallen into the snow:

"Due to newer security protocols, please use password method 0x434C49434B until further notice."

You remember from the training seminar that "method 0x434C49434B" means you're actually supposed to count the number of times any click causes the dial to point at 0, regardless of whether it happens during a rotation or at the end of one.

Following the same rotations as in the above example, the dial points at zero a few extra times during its rotations:

The dial starts by pointing at 50.

The dial is rotated L68 to point at 82; during this rotation, it points at 0 once.

The dial is rotated L30 to point at 52.

The dial is rotated R48 to point at 0.

The dial is rotated L5 to point at 95.

The dial is rotated R60 to point at 55; during this rotation, it points at 0 once.

The dial is rotated L55 to point at 0.

The dial is rotated L1 to point at 99.

The dial is rotated L99 to point at 0.

The dial is rotated R14 to point at 14.

The dial is rotated L82 to point at 32; during this rotation, it points at 0 once.

In this example, the dial points at 0 three times at the end of a rotation, plus three more times during a rotation. So, in this example, the new password would be 6.

Be careful: if the dial were pointing at 50, a single rotation like R1000 would cause the dial to point at 0 ten times before returning back to 50!

Using password method 0x434C49434B, what is the password to open the door?

In [82]:
def zeroes_count_0x434C49434B(start, numbers, target):
    current = start
    num_of_occurrences = 0
    for next in numbers:
        full_rotations = abs(next) // 100
        num_of_occurrences += full_rotations 
        difference = next % 100 if next >= 0 else -(-next % 100)
        if did_we_cross_zero(current, difference):
            num_of_occurrences += 1
        current = (current + difference) % 100 if (current + difference) >= 0 else -(-(current + difference) % 100)
        if current == target:
            num_of_occurrences += 1
        logger.debug(f"After processing {next}, current: {current}, total times passed or hit 0: {num_of_occurrences}\n")
    return num_of_occurrences

def did_we_cross_zero(current, difference):
    logger.debug(f"Current: {current}, Difference: {difference}")
    if (current + difference) * current < 0:
        logger.debug("Crossed zero due to sign change")
        return True
    if abs(current + difference) > 100:
        logger.debug("Crossed zero due to exceeding 100")
        return True
    return False

In [83]:
logger.setLevel(logging.DEBUG)
test = [-68, -30, 48, -5, 60, -55, -1, -99, 14, -82]
test_solution = 6
zeroes_count_0x434C49434B(50, test, 0)

DEBUG:__main__:Current: 50, Difference: -68
DEBUG:__main__:Crossed zero due to sign change
DEBUG:__main__:After processing -68, current: -18, total times passed or hit 0: 1

DEBUG:__main__:Current: -18, Difference: -30
DEBUG:__main__:After processing -30, current: -48, total times passed or hit 0: 1

DEBUG:__main__:Current: -48, Difference: 48
DEBUG:__main__:After processing 48, current: 0, total times passed or hit 0: 2

DEBUG:__main__:Current: 0, Difference: -5
DEBUG:__main__:After processing -5, current: -5, total times passed or hit 0: 2

DEBUG:__main__:Current: -5, Difference: 60
DEBUG:__main__:Crossed zero due to sign change
DEBUG:__main__:After processing 60, current: 55, total times passed or hit 0: 3

DEBUG:__main__:Current: 55, Difference: -55
DEBUG:__main__:After processing -55, current: 0, total times passed or hit 0: 4

DEBUG:__main__:Current: 0, Difference: -1
DEBUG:__main__:After processing -1, current: -1, total times passed or hit 0: 4

DEBUG:__main__:Current: -1, Diff

6

In [84]:
logger.setLevel(logging.INFO)
input_split = input.splitlines()
input_numbers = [int(x[1:]) if x[0] == 'R' else -int(x[1:]) for x in input_split]
solution = zeroes_count_0x434C49434B(50, input_numbers, 0)
print(f"The solution is {solution}.")

The solution is 6305.
